In [1]:
%load_ext Cython

In [2]:
%%cython

import os
import os.path

arrayfile = []
line = 1
file1=0

cdef classvardicFunc():
    global arrayfile
    global line
    global file1
    file1.write("<classVarDec>\n")
    file1.write(arrayfile[line])
    line= line+1
    file1.write(arrayfile[line])
    line=line+1
    file1.write(arrayfile[line])
    line=line+1
    while ((line < len(arrayfile)) and ("," in arrayfile[line])):
        file1.write(arrayfile[line])
        line=line+1
        file1.write(arrayfile[line])
        line=line+1
    file1.write(arrayfile[line])
    file1.write("</classVarDec>\n")

cdef varfunc():
    global arrayfile
    global line
    global file1
    file1.write("<varDec>\n")
    file1.write(arrayfile[line])
    line= line+1
    file1.write(arrayfile[line])
    line=line+1
    file1.write(arrayfile[line])
    line=line+1
    while ((line < len(arrayfile)) and ("," in arrayfile[line])):
        file1.write(arrayfile[line])
        line=line+1
        file1.write(arrayfile[line])
        line=line+1
    file1.write(arrayfile[line])
    file1.write("</varDec>\n")
    
####### termFunc #########
cdef termFunc():
    global arrayfile
    global line
    global file1
    flag=True
    while (line+1< len(arrayfile) and("integerConstant" in arrayfile[line+1] 
                                  or "stringConstant" in arrayfile[line+1] 
                                  or"this" in arrayfile[line+1] or "true" in arrayfile[line+1] 
                                  or "false" in arrayfile[line+1] or "null" in arrayfile[line+1]
                                  or "identifier" in arrayfile[line+1] or "(" in arrayfile[line+1] 
                                   or "-" in arrayfile[line+1] or "~" in arrayfile[line+1])):
        file1.write("<term>\n")
        line=line+1
        file1.write(arrayfile[line])
        if ("integerConstant" in arrayfile[line] or "stringConstant" in arrayfile[line]
           or "this" in arrayfile[line] or "true" in arrayfile[line]
            or "false" in arrayfile[line] or "null" in arrayfile[line]):
            file1.write("</term>\n")
        elif "(" in arrayfile[line]:
            expressionfunc()
            line=line+1
            file1.write(arrayfile[line])#)
            file1.write("</term>\n")
        elif "identifier" in arrayfile[line]:
            if "[" in arrayfile[line+1]:
                line=line+1
                file1.write(arrayfile[line])
                expressionfunc()
                line=line+1
                file1.write(arrayfile[line])#]
                file1.write("</term>\n")
            elif "(" in arrayfile[line+1] or "." in arrayfile[line+1]:
                subroutineCallFunc()
                file1.write("</term>\n")
            else:
                file1.write("</term>\n")
        elif  "~" in arrayfile[line] or "-" in arrayfile[line]:
            flag=False
        temp=arrayfile[line+1].split(" ")
        if  ("~" in temp[1] or "+" in temp[1] or "-" in temp[1] or "*" in temp[1] 
             or "/" in temp[1] 
             or "&" in temp[1] 
             or "|" in temp[1] 
             or "&lt" in temp[1]
             or "&gt" in temp[1] 
             or "=" in temp[1]):
            line=line+1
            file1.write(arrayfile[line])#op 
    if not(flag):
        file1.write("</term>\n")
       
 
    
  ###### expressionfunc  ##########          
cdef expressionfunc():    
    global arrayfile
    global line
    global file1 
    if (line+1< len(arrayfile) and("integerConstant" in arrayfile[line+1] 
                                  or "stringConstant" in arrayfile[line+1] 
                                  or"this" in arrayfile[line+1] or "true" in arrayfile[line+1] 
                                  or "false" in arrayfile[line+1] or "null" in arrayfile[line+1]
                                  or "identifier" in arrayfile[line+1] or "(" in arrayfile[line+1] 
                                   or "-" in arrayfile[line+1] or "~" in arrayfile[line+1])):
        file1.write("<expression>\n")
        termFunc()
        file1.write("</expression>\n")
    
    
  

    
    
###### expressionListfunc  ##########  
cdef expressionListfunc():
    global arrayfile
    global line
    global file1 
    if(line+1< len(arrayfile) and("integerConstant" in arrayfile[line+1] 
                                  or "stringConstant" in arrayfile[line+1] 
                                  or"this" in arrayfile[line+1] or "true" in arrayfile[line+1] 
                                  or "false" in arrayfile[line+1] or "null" in arrayfile[line+1]
                                  or "identifier" in arrayfile[line+1] or "(" in arrayfile[line+1] 
                                   or "-" in arrayfile[line+1] or "~" in arrayfile[line+1])):
        file1.write("<expressionList>\n")                  
        line=line+1
        expressionfunc()
        line=line+1
        while line < len(arrayfile) and "," in arrayfile[line]:
            file1.write(arrayfile[line])
            expressionfunc()
            line=line+1
        file1.write("</expressionList>\n")
    else:
        line=line+1    
        
####### subroutineCallFunc ########        
cdef subroutineCallFunc():
    global arrayfile
    global line
    global file1 
    line=line+1
    file1.write(arrayfile[line])#( .
    if "(" in arrayfile[line]:
        line=line-1
        expressionListfunc()
        file1.write(arrayfile[line])#)
    if "." in arrayfile[line]:
        line=line+1
        file1.write(arrayfile[line])#id -subroutin
        line=line+1
        file1.write(arrayfile[line])#(
        line=line-1
        expressionListfunc()
        file1.write(arrayfile[line])#)
        
        
#######  LET  ##########       
cdef letfunc():
    global arrayfile
    global line
    global file1
    file1.write("<letStatement>\n")
    file1.write(arrayfile[line])#let
    line=line+1
    file1.write(arrayfile[line])#id
    line=line+1
    if "[" in arrayfile[line]:
        file1.write(arrayfile[line])
        expressionfunc()
        line=line+1
        file1.write(arrayfile[line])#]
        line=line+1
    file1.write(arrayfile[line]) #=
    expressionfunc()
    line=line+1
    file1.write(arrayfile[line])#;
    file1.write("</letStatement>\n")
    
#######  DO  ##########
cdef dofunc():
    global arrayfile
    global line
    global file1
    file1.write("<doStatement>\n")
    file1.write(arrayfile[line])
    line=line+1
    file1.write(arrayfile[line])
    subroutineCallFunc()
    line=line+1
    file1.write(arrayfile[line])#;
    file1.write("</doStatement>\n")

#######  IF  ##########
cdef ifFunc():
    global arrayfile
    global line
    global file1
    file1.write("<ifStatement>\n")
    file1.write(arrayfile[line])
    line=line+1
    file1.write(arrayfile[line])#(
    expressionfunc()
    line=line+1
    file1.write(arrayfile[line])#)
    line=line+1
    file1.write(arrayfile[line])#{
    line=line+1
    statement()
    file1.write(arrayfile[line])           
    if("else" in arrayfile[line+1]):
        line=line+1
        file1.write(arrayfile[line])
        line=line+1
        file1.write(arrayfile[line])
        line=line+1  
        statement()
        file1.write(arrayfile[line]) 
    file1.write("</ifStatement>\n")

                
cdef subroutineDecFunc():
    global arrayfile
    global line
    global file1
    file1.write("<subroutineDec>\n")
    file1.write(arrayfile[line])
    line= line+1
    file1.write(arrayfile[line])
    line=line+1
    file1.write(arrayfile[line])
    line=line+1
    file1.write(arrayfile[line])
    line=line+1
    file1.write("<parameterList>\n")
    while ((line < len(arrayfile)) and not(")" in arrayfile[line])):
        file1.write(arrayfile[line])
        line=line+1
    file1.write("</parameterList>\n")
    file1.write(arrayfile[line])
    line=line+1
    file1.write("<subroutineBody>\n")
    file1.write(arrayfile[line])
    line=line+1
    while line < len(arrayfile) and "var" in arrayfile[line]:
        varfunc()
        line=line+1
    statement()
    file1.write(arrayfile[line])
    file1.write("</subroutineBody>\n")
    file1.write("</subroutineDec>\n")


####### returnfunc ##########
cdef returnfunc():
    global arrayfile
    global line
    global file1
    file1.write("<returnStatement>\n")
    file1.write(arrayfile[line])
    expressionfunc()
    line=line+1
    file1.write(arrayfile[line])
    file1.write("</returnStatement>\n")
    
######## WHILE ############
cdef whilefunc():
    global arrayfile
    global line
    global file1
    file1.write("<whileStatement>\n")
    file1.write(arrayfile[line])
    line=line+1
    file1.write(arrayfile[line])#(
    expressionfunc()
    line=line+1
    file1.write(arrayfile[line])
    line=line+1
    file1.write(arrayfile[line])
    line=line+1
    statement()
    file1.write(arrayfile[line])        
    file1.write("</whileStatement>\n")   
    
############### statement ####################   
cdef statement():
    global arrayfile
    global line
    global file1
    file1.write("<statements>\n")
    while line< len(arrayfile) and ("let" in arrayfile[line] or "do" in arrayfile[line] or "while" in arrayfile[line] or " if " in arrayfile[line]):
        if "let" in arrayfile[line]:
            letfunc()
        elif "do" in arrayfile[line]:
            dofunc()
        elif "while" in arrayfile[line]:
            whilefunc()
        elif " if " in arrayfile[line]:
            ifFunc()
        line=line+1
    if "return" in arrayfile[line]:
        returnfunc()
        line=line+1
    file1.write("</statements>\n")
    



#MAIN
################################
path = input()
print("\n")
global arrayfile
global line
global file1
for filenames in os.listdir(path):
    if("T.xml" in filenames):
        with open(os.path.join(path, filenames)) as file:
            strt=(file.name).split("\\")[-1]    
            if(".xml" in file.name):
                line=1
                str = (filenames).split(".")
                temp = path+"\\"
                file1 = open(temp+str[0][:-1]+".xml", "w")
                arrayfile = file.readlines()  
                file1.write("<class>\n")
                file1.write(arrayfile[line])
                line = line + 1
                file1.write(arrayfile[line])
                line = line + 1
                file1.write(arrayfile[line])
                line = line + 1
                while line < len(arrayfile)-2:
                    if (("static" in arrayfile[line]) or ("field" in arrayfile[line])):
                        classvardicFunc()
                    if ("constructor" in arrayfile[line]) or ("method" in arrayfile[line]) or ("function" in arrayfile[line]):
                        subroutineDecFunc()
                    line = line + 1
            file1.write("<symbol> } </symbol>\n")
            #file1.write(arrayfile[line])
            file1.write("</class>\n")
            file1.close()


C:\Users\איילת\Desktop\עקרונות שפות תוכנה\Exercises\Targil4\project 10\ArrayTest - Copy\TP


